# interactive graph cutting in napari

In [6]:
from grace.utils import ui
from grace.datasets import random_graph

In [7]:
import napari
import numpy as np

In [8]:
G = random_graph(scale=1000)

In [9]:
points, edges = ui.graph_to_napari_layers(G)

In [10]:
viewer = napari.Viewer()
viewer.add_shapes(edges, shape_type="line", edge_width=1)
gl = viewer.add_shapes(
    np.empty((0, 2, 2)), 
    shape_type="line", 
    edge_color="green", 
    edge_width=2, 
    name="good"
)
bl = viewer.add_shapes(
    np.empty((0, 2, 2)), 
    shape_type="line", 
    edge_color="red", 
    edge_width=2, 
    name="bad"
)
viewer.add_points(points)
viewer.add_labels(np.zeros((1000, 1000), dtype=np.int32), name="mask")
viewer.layers["mask"].brush_size = 20

# press `k` after drawing in the `mask` layer to cut the graph
@viewer.bind_key("k")
def cut(e):
    idx, good, bad = ui.cut_graph_using_mask(G, viewer.layers["mask"].data)

    good_arr = np.array(
        [points[(i, j), :] for i, j in good]
    )

    bad_arr = np.array(
        [points[(i, j), :] for i, j in bad]
    )

    gl.data = [] 
    bl.data = []
    gl.add_lines(good_arr)
    bl.add_lines(bad_arr)
